In [ ]:
# import pandas
# train_data = pandas.read_pickle('train.pkl.gz')
# test_data = pandas.read_pickle('test.pkl.gz')

In [ ]:
#google drive 연동
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
#데이터 로드 부분 제외

## data exploration


In [ ]:
print("train data index unique nums :", train_data["index"].nunique() )

print("test data index unique nums :", test_data["index"].nunique() )

train data index unique nums : 1253
test data index unique nums : 1253


In [ ]:
# print("train data index unique array :", train_data["index"].unique() )

# print("test data index unique array :", test_data["index"].unique() )

## data preprocessing

In [ ]:
#날짜-시간 전처리 #2202 -> 이부분 계산 너무 오래 걸림..
from datetime import datetime
dateFormatter = "%Y-%m-%d %H:%M:%S"

for i in range(len(train_data["dt"])):
  converted_datetime = train_data["dt"][i]#datetime.strptime(train_data["dt"][i], dateFormatter)
  train_data["dt"][i] =  converted_datetime.timestamp()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
for i in range(len(test_data["dt"])):
  converted_datetime = test_data["dt"][i]#datetime.strptime(test_data["dt"][i], dateFormatter)
  test_data["dt"][i] =  converted_datetime.timestamp()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## feature encoding

In [ ]:
#location(pos) encoding
from sklearn.preprocessing import LabelEncoder


pos_encoder= LabelEncoder()
pos_encoder.fit(train_data["index"])
train_data["index"] = pos_encoder.transform(train_data["index"])



In [ ]:
pos_encoder.fit(test_data["index"])
test_data["index"] = pos_encoder.transform(test_data["index"])



In [ ]:
train = train_data
test = test_data


In [ ]:
import pandas
test = pandas.read_csv("/content/drive/MyDrive/data/" + "test_data2.csv")
train = pandas.read_csv("/content/drive/MyDrive/data/"+ "train_data2.csv")

In [ ]:
test = test.drop(test.columns[0], axis=1)
train = train.drop(train.columns[0], axis=1)



In [ ]:
train_data = train
test_data = test


In [ ]:
test_data.shape #(229451, 3)

(229451, 3)

In [ ]:
grouped = train.groupby("index")
train_data_D = {}
for key, group in grouped:
  if train_data_D.get(  key ) is None:
    train_data_D[key] = group
  #print(key)
  #print(group.head())

In [ ]:
grouped = test.groupby("index")
test_data_D = {}
for key, group in grouped:
  if test_data_D.get(  key ) is None:
    test_data_D[key] = group

In [ ]:
for key in train_data_D.keys():
  train_data_D[key] = train_data_D[key].values
print(train_data_D[0].shape) #(1323, 3)

(1323, 3)


In [ ]:
for key in test_data_D.keys():
  test_data_D[key] = test_data_D[key].values
print(test_data_D[0].shape) #(185, 3)

(185, 3)


## modeling

In [ ]:
import numpy as np

def split_xy6(dataset, time_steps, y_column):
  x, y = list(), list()
  for i in range( len(dataset)):
    x_end_number = i + time_steps
    y_end_number = x_end_number + y_column

    if y_end_number > len(dataset):     break

    tmp_x = dataset[i:x_end_number, :]
    tmp_y = dataset[x_end_number:y_end_number, 2]

    x.append(tmp_x)
    y.append(tmp_y)
  return np.array(x), np.array(y)


    

In [ ]:
models = []


In [ ]:
test_data_groups = []

In [ ]:
from tensorflow.keras.models import Sequential #배치 노멀라이제이션 추가해보려고함, es, epoch, bs 바꿔줌(3, 20, 1 -> 20, 300, 128)
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping
for i in range(len(train_data_D) ):
  print(" %d th model in %d "%(i, len(train_data_D)))
  x, y = split_xy6(train_data_D[i], 7, 6) # 0 - 1252
  x_test, y_test = split_xy6(test_data_D[i], 7, 6)


  x_train = np.reshape(x, 
                      (x.shape[0], x.shape[1] * x.shape[2]))
  x_test = np.reshape(x_test,
                      (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))
  
  
  scaler = StandardScaler()        #MinmaxScaler 해볼것
  scaler.fit(x_train)
  x_train_scaled = scaler.transform(x_train)
  x_test_scaled = scaler.transform(x_test)
  

  test_data_groups.append( [ x_test_scaled, y_test ]   )

  model_scaled = Sequential()
  model_scaled.add(Dense(64, input_shape = (21, )))
  model_scaled.add(BatchNormalization())
  model_scaled.add(Dense(32, activation = 'relu'))
  model_scaled.add(BatchNormalization())
  model_scaled.add(Dense(32, activation = 'relu'))
  model_scaled.add(BatchNormalization())
  model_scaled.add(Dense(32, activation = 'relu'))
  model_scaled.add(BatchNormalization())
  model_scaled.add(Dense(32, activation = 'relu'))
  model_scaled.add(BatchNormalization())
  model_scaled.add(Dense(6))

  model_scaled.compile(loss='mse', optimizer='adam', metrics=['mse'])

  
  early_stopping = EarlyStopping(patience=20) #20

  model_scaled.fit(x_train_scaled, y, validation_split=0.2, verbose=1, 
            batch_size=128, epochs=300, callbacks=[early_stopping]) #50
 
  models.append(model_scaled)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1053/1053 [==============================] - 2s 2ms/step - loss: 178.1813 - mse: 178.1813 - val_loss: 81.2431 - val_mse: 81.2431
 791 th model in 1253 
Epoch 1/5
1417/1417 [==============================] - 3s 2ms/step - loss: 4566.3237 - mse: 4566.3237 - val_loss: 3028.5679 - val_mse: 3028.5679
Epoch 2/5
1417/1417 [==============================] - 3s 2ms/step - loss: 3079.1936 - mse: 3079.1936 - val_loss: 1661.0485 - val_mse: 1661.0485
Epoch 3/5
1417/1417 [==============================] - 3s 2ms/step - loss: 2405.9597 - mse: 2405.9597 - val_loss: 2682.5952 - val_mse: 2682.5952
Epoch 4/5
1417/1417 [==============================] - 3s 2ms/step - loss: 2244.3870 - mse: 2244.3870 - val_loss: 5180.6577 - val_mse: 5180.6577
Epoch 5/5
1417/1417 [==============================] - 3s 2ms/step - loss: 1992.9326 - mse: 1992.9326 - val_loss: 3408.0520 - val_mse: 3408.0520
 792 th model in 1253 
Epoch 1/5
445/445 [==============================] - 1s 2ms/step

In [ ]:
models[0]

## evaluation



In [ ]:
#test_data_groups[i]

mse_sums = 0
mse_lens = 0

for i in range(len(models)):
  model = models[i]

  loss, mse =  model.evaluate(test_data_groups[i][0], test_data_groups[i][1], batch_size=1)
  print('loss : ',loss)
  print('mse : ' , mse)

  mse_lens += len( test_data_groups[i][0] )
  mse_sums += (len( test_data_groups[i][0] ) * mse)


173/173 [==============================] - 0s 2ms/step - loss: 33.2954 - mse: 33.2954
loss :  33.29543685913086
mse :  33.29543685913086
184/184 [==============================] - 0s 2ms/step - loss: 35.7260 - mse: 35.7260
loss :  35.726043701171875
mse :  35.726043701171875
208/208 [==============================] - 0s 1ms/step - loss: 185.8927 - mse: 185.8927
loss :  185.89271545410156
mse :  185.89271545410156
83/83 [==============================] - 0s 1ms/step - loss: 15.1051 - mse: 15.1051
loss :  15.105100631713867
mse :  15.105100631713867
288/288 [==============================] - 0s 1ms/step - loss: 1594.5723 - mse: 1594.5723
loss :  1594.572265625
mse :  1594.572265625
317/317 [==============================] - 0s 1ms/step - loss: 171.7717 - mse: 171.7717
loss :  171.77174377441406
mse :  171.77174377441406
209/209 [==============================] - 0s 1ms/step - loss: 141.8401 - mse: 141.8401
loss :  141.84011840820312
mse :  141.84011840820312
178/178 [====================

In [ ]:
print("total mse : ", mse_sums / mse_lens) #total mse :  172203.2085991607 기존

total mse :  172203.2085991607


In [ ]:
#test_data_groups[i]

mse_sums = 0
mse_lens = 0

for i in range(len(models)):
  model = models[i]

  loss, mse =  model.evaluate(test_data_groups[i][0], test_data_groups[i][1], batch_size=1)
  print('loss : ',loss)
  print('mse : ' , mse)

  mse_lens += len( test_data_groups[i][0] )
  mse_sums += (len( test_data_groups[i][0] ) * mse)

173/173 [==============================] - 0s 1ms/step - loss: 33.2954 - mse: 33.2954
loss :  33.29543685913086
mse :  33.29543685913086
184/184 [==============================] - 0s 1ms/step - loss: 35.7260 - mse: 35.7260
loss :  35.726043701171875
mse :  35.726043701171875
208/208 [==============================] - 0s 1ms/step - loss: 185.8927 - mse: 185.8927
loss :  185.89271545410156
mse :  185.89271545410156
83/83 [==============================] - 0s 1ms/step - loss: 15.1051 - mse: 15.1051
loss :  15.105100631713867
mse :  15.105100631713867
288/288 [==============================] - 0s 1ms/step - loss: 1594.5723 - mse: 1594.5723
loss :  1594.572265625
mse :  1594.572265625
317/317 [==============================] - 0s 1ms/step - loss: 171.7717 - mse: 171.7717
loss :  171.77174377441406
mse :  171.77174377441406
209/209 [==============================] - 0s 2ms/step - loss: 141.8401 - mse: 141.8401
loss :  141.84011840820312
mse :  141.84011840820312
178/178 [====================

In [ ]:
print("total mse : ", mse_sums / mse_lens) #total mse :  431.54246634721915 기존

total mse :  431.54246634721915


In [ ]:
#test_data_groups[i]

mse_sums = 0
mse_lens = 0

for i in range(len(models)):
  model = models[i]

  loss, mse =  model.evaluate(test_data_groups[i][0], test_data_groups[i][1], batch_size=1)
  print('loss : ',loss)
  print('mse : ' , mse)

  mse_lens += len( test_data_groups[i][0] )
  mse_sums += mse

173/173 [==============================] - 0s 1ms/step - loss: 33.2954 - mse: 33.2954
loss :  33.29543685913086
mse :  33.29543685913086
184/184 [==============================] - 0s 1ms/step - loss: 35.7260 - mse: 35.7260
loss :  35.726043701171875
mse :  35.726043701171875
208/208 [==============================] - 0s 1ms/step - loss: 185.8927 - mse: 185.8927
loss :  185.89271545410156
mse :  185.89271545410156
83/83 [==============================] - 0s 1ms/step - loss: 15.1051 - mse: 15.1051
loss :  15.105100631713867
mse :  15.105100631713867
288/288 [==============================] - 0s 1ms/step - loss: 1594.5723 - mse: 1594.5723
loss :  1594.572265625
mse :  1594.572265625
317/317 [==============================] - 0s 1ms/step - loss: 171.7717 - mse: 171.7717
loss :  171.77174377441406
mse :  171.77174377441406
209/209 [==============================] - 0s 1ms/step - loss: 141.8401 - mse: 141.8401
loss :  141.84011840820312
mse :  141.84011840820312
178/178 [====================

In [ ]:
print("average mse : ", mse_sums / len(models) ) #average mse :  273.0679375855141 기존

average mse :  273.0679375855141


In [ ]:
i=795

x, y = split_xy6(train_data_D[i], 7, 6) # 0 - 1252
x_test, y_test = split_xy6(test_data_D[i], 7, 6)


x_train = np.reshape(x, 
                    (x.shape[0], x.shape[1] * x.shape[2]))
x_test = np.reshape(x_test,
                    (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))


scaler = StandardScaler()        #MinmaxScaler 해볼것
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
#전처리

scaler = StandardScaler()        #MinmaxScaler 해볼것
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
#다시 전처리

x_train_scaled = np.reshape(x_train_scaled, 
                            (x_train_scaled.shape[0],  7, 3)
                            )   #scaler.transform(x_train)
x_test_scaled = np.reshape(x_test_scaled,
                           (x_test_scaled.shape[0],  7, 3)    
                          )#scaler.transform(x_test)

In [ ]:
x_train_scaled.shape

(2037, 7, 3)

In [ ]:
x_test_scaled.shape

(319, 7, 3)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

model_LSTM = Sequential()
model_LSTM.add(LSTM(64, input_shape = (7, 3)))
model_LSTM.add(Dense(32, activation = 'relu'))
model_LSTM.add(Dense(32, activation = 'relu'))
model_LSTM.add(Dense(32, activation = 'relu'))
model_LSTM.add(Dense(32, activation = 'relu'))
model_LSTM.add(Dense(6))

model_LSTM.compile(loss='mse', optimizer='adam', metrics=['mse'])


In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=20)

model_LSTM.fit(x_train_scaled, y, validation_split=0.2, verbose=1, 
          batch_size=1, epochs=100, callbacks=[early_stopping])


Epoch 1/100
1629/1629 [==============================] - 7s 4ms/step - loss: 1225.9998 - mse: 1225.9998 - val_loss: 815.5091 - val_mse: 815.5091
Epoch 2/100
1629/1629 [==============================] - 6s 3ms/step - loss: 724.4222 - mse: 724.4222 - val_loss: 765.3546 - val_mse: 765.3546
Epoch 3/100
1629/1629 [==============================] - 6s 4ms/step - loss: 635.8600 - mse: 635.8600 - val_loss: 644.8109 - val_mse: 644.8109
Epoch 4/100
1629/1629 [==============================] - 6s 4ms/step - loss: 596.5964 - mse: 596.5964 - val_loss: 523.6168 - val_mse: 523.6168
Epoch 5/100
1629/1629 [==============================] - 6s 4ms/step - loss: 555.0717 - mse: 555.0717 - val_loss: 568.2416 - val_mse: 568.2416
Epoch 6/100
1629/1629 [==============================] - 6s 4ms/step - loss: 565.1490 - mse: 565.1490 - val_loss: 512.1378 - val_mse: 512.1378
Epoch 7/100
1629/1629 [==============================] - 6s 4ms/step - loss: 532.0623 - mse: 532.0623 - val_loss: 565.2619 - val_mse: 565.26

In [ ]:
loss, mse =  model_LSTM.evaluate(x_test_scaled, y_test, batch_size=1)
print('loss : ',loss)
print('mse : ' , mse)

319/319 [==============================] - 1s 2ms/step - loss: 516.5095 - mse: 516.5095
loss :  516.5094604492188
mse :  516.5094604492188
